In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

In [ ]:
def yolo2bbox(box):
    x_min, y_min = box[0] - box[2]/2, box[1] - box[3]/2
    x_max, y_max = box[0] + box[2]/2, box[1] + box[3]/2
    return x_min, y_min, x_max, y_max


def plot_box(image, boxes, labels):
    img_h, img_w, _ = image.shape
    for i, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = yolo2bbox(box)
        x_1 = int(x_min * img_w)
        y_1 = int(y_min * img_h)
        x_2 = int(x_max * img_w)
        y_2 = int(y_max * img_h)
        width = x_2 - x_1
        height = y_2 - y_1
        class_name = class_names[int(labels[i])]
        color = colors[class_names.index(class_name)]
        cv2.rectangle(image, (x_1, y_1), (x_2, y_2), color=color, thickness=2)
        
        font_scale = min(1, max(3, int(img_w/500)))
        font_thickness = min(2, max(10, int(img_w/50)))
        tw, th = cv2.getTextSize(class_name, 0, fontScale=font_scale, thickness=font_thickness)[0]
        p1, p2 = (int(x_1), int(y_1)), (int(x_2), int(y_2))
        p2 = p1[0] + tw, p1[1] + -th - 10
        cv2.rectangle(image, p1, p2, color=color, thickness=-1,)
        cv2.putText(image, class_name, (x_1+1, y_1-10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255),
        font_thickness)
    return image



def plot(images_path, labels_path, display_qty):
    training_images = glob.glob(images_path)
    training_labels = glob.glob(labels_path)
    training_images.sort()
    training_labels.sort()
    
    images_qty = len(training_images)
    
    plt.figure(figsize=(15,12))
    for i in range(display_qty):
        j = random.randint(0, images_qty-1)
        image = cv2.imread(training_images[j])
        boxes = []
        labels = []
        with open(training_labels[j], 'r') as f:
            textfile_lines = f.readlines()
            for textfile_line in textfile_lines:
                label = textfile_line.split(' ')[0]
                x_c, y_c, w, h = textfile_line.split(' ')[1:]
                x_c = float(x_c)
                y_c = float(y_c)
                w   = float(w)
                h   = float(h.split('\n')[0])
                boxes.append([x_c, y_c, w, h])
                labels.append(label)
        plot_box(image, boxes, labels)
        plt.subplot(3,3, i+1)
        im_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(im_rgb)
        plt.axis('off')
    

# class_names = ['F20_20_B', 'R20', 'S40_40_B', 'S40_40_G', 'axis', 'bearing_box', 'bracket', 'brown_box', 'cup', 
#                'dishwasher_soap', 'eye_glasses', 'insulation_tape', 'motor', 'pringles', 'screw_driver', 'sponge', 
#                'spoon', 'tennis_ball', 'toothbrush', 'towel']

# image_paths = 'dataset/train1/images/*'
# label_paths = 'dataset/train1/labels/*'



class_names = ['F20_20_B', 'R20', 'S40_40_B', 'S40_40_G', 'axis', 'bearing_box', 'bracket', 'brown_box', 'cup', 
               'dishwasher_soap', 'eye_glasses', 'insulation_tape', 'motor', 'pringles', 'screw_driver', 'sponge', 
               'spoon', 'tennis_ball', 'toothbrush', 'towel', "cereal","chocolate_milk", "heineken","iron_man" ,"medicine"
               ,"milk_bottle","milk_box","monster" ,"purple_juice" ,"red_juice" ,"shampoo" ,"tea_box", "yellow_juice", "paprika"]

image_paths = 'dataset/test300/images/*'
label_paths = 'dataset/test300/labels/*'


colors = np.random.uniform(0, 255, size=(len(class_names), 3))
plot(image_paths, label_paths,6)

In [ ]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

def SetResultDirectory():
    global Folder_count
    Folder_count = len(glob.glob("runs/train/*"))
    if Train:
        DirectoryName = "./results_" + str(Folder_count+1)
    else:
        DirectoryName = "./results_" + str(Folder_count)
    return DirectoryName


Train  = True
Freeze = True
Epochs = 100
Folder_count = 0
%cd yolov5
Result_Directory = SetResultDirectory()

In [ ]:
if Train:
    if Freeze:
        !python -m torch.distributed.run --nproc_per_node 1 train.py \
        --batch  4\
        --data ../dataset/MergedDataset200.yaml \
        --weights yolov5m.pt \
        --img 640 \
        --epochs {Epochs} \
        --name {Result_Directory} \
        --device 1 \
        --freeze 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 \
#         --hyp data/hyps/hyp.no-augmentation.yaml 
    else:
        !python -m torch.distributed.run --nproc_per_node 2 train.py \
        --batch  4\
        --data ../dataset/MergedDataset200.yaml \
        --weights yolov5m.pt \
        --img 640 \
        --epochs {Epochs} \
        --name {Result_Directory} \
        --device 0,1 \
        
    Result_Directory = "runs/train/results_" + str(Folder_count + 1)
    
else:
    Result_Directory = "runs/train/results_" + str(Folder_count)

In [ ]:
def show_valid_results(Result_Directory):
    validation_pred_images = glob.glob(Result_Directory + "/*_pred.jpg")
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

show_valid_results(Result_Directory)

In [ ]:
def inference(Result_Directory, data_path):
    infer_folder_count = len(glob.glob('runs/detect/*'))
    Detect_store = f"./inference_{infer_folder_count+1}"
    Best_weights = f"{Result_Directory}/weights/best.pt"
    
    !python -m torch.distributed.run --nproc_per_node 1 detect.py --weights {Best_weights} \
    --source {data_path} \
    --name {Detect_store}\
    --device 1
    
    return Detect_store


on_single_image = False
if on_single_image:
    IMAGE_INFER_DIR = inference(Result_Directory, '../dataset/test200/Test_Image.jpg')
else:
    IMAGE_INFER_DIR = inference(Result_Directory, '../dataset/test200/Multiple_images')

# if on_single_image:
#     IMAGE_INFER_DIR = inference(Result_Directory, '../dataset/test1/Test_Image.jpg')
# else:
#     IMAGE_INFER_DIR = inference(Result_Directory, '../dataset/test1/Multiple_images')

In [ ]:
def visualize(INFER_DIR):
    INFER_PATH = f"{INFER_DIR}"
    infer_images = glob.glob(f"{INFER_PATH}/*")
    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()
        
visualize('runs/detect/' + IMAGE_INFER_DIR[2:])

In [ ]:
# infer_folder_count = len(glob.glob('runs/detect/*'))
# Detect_store = f"./inference_{infer_folder_count+1}"
# Best_weights = f"{Result_Directory}/weights/best.pt"

# !python val.py --weights runs/train/results_merged3/weights/best.pt \
# --data ../dataset/TestMergeDataset100.yaml

In [ ]:
# %cd yolov5
#!python export.py --weights /home/jovyan/public/mnadar/result/results_0/weights/best.pt --include onnx
# !python export.py --weights /home/jovyan/public/mnadar/result/results_0/weights/best.pt --include edgetpu
#!python export.py --weights /home/jovyan/public/mnadar/result/results_0/weights/best.pt --include tflite

In [ ]:
def inference():
    infer_folder_count = len(glob.glob('runs/detect/*'))
    Detect_store = f"./inference_{infer_folder_count+1}"
    
    Result_Directory = "runs/train/results_merged" + str(1)
    Best_weights = f"{Result_Directory}/weights/best.pt"
    
    !python -m torch.distributed.run --nproc_per_node 1 detect.py --weights {Best_weights} \
    --source ../dataset/final_test \
    --name {Detect_store}\
    --device 1

inference()